In [9]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

import os
import itertools
from glob import glob
import PIL
from PIL import Image
from scipy import ndimage 
import scipy
import sys

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout

# dataframe  id  label  image-array

base_crack_Negative = os.path.join("c:/", "Users", "srija", "Downloads", "concrete_crack_images","mlops", "concerete_crack_images", "training", "Negative")
base_crack_Positive = os.path.join("c:/", "Users", "srija", "Downloads", "concrete_crack_images","mlops", "concerete_crack_images", "training", "Positive")

crack_Negative_df = pd.DataFrame(columns=( 'path', 'image', 'label'),index=np.arange(0,16037))
crack_Positive_df = pd.DataFrame(columns=( 'path', 'image', 'label'),index=np.arange(0,16070))

imageid_path_dict_Negative = np.array([x for x in glob(os.path.join(base_crack_Negative, '*.jpg'))])
imageid_path_dict_Positive = np.array([x for x in glob(os.path.join(base_crack_Positive, '*.jpg'))])

In [8]:
crack_Negative_df['path']=imageid_path_dict_Negative
crack_Negative_df['label']=0
crack_Negative_df['image']=crack_Negative_df['path'].map(lambda x: np.asarray(Image.open(x).resize((75,100))))

crack_Positive_df['path']=imageid_path_dict_Positive
crack_Positive_df['label']=1
crack_Positive_df['image']=crack_Positive_df['path'].map(lambda x: np.asarray(Image.open(x).resize((75,100))))

print("Negative:",crack_Negative_df["image"][0].shape)
print("Positive:",crack_Positive_df["image"][0].shape)

print(imageid_path_dict_Negative.shape,imageid_path_dict_Positive.shape)

crack_df = crack_Negative_df.append(crack_Positive_df)  
crack_df.reset_index(drop=True,inplace=True) 
print(crack_df.shape)

Negative: (100, 75, 3)
Positive: (100, 75, 3)
(16037,) (16070,)
(32107, 3)


In [10]:
features=crack_df.drop(columns=['label'],axis=1)
target=crack_df["label"]

x_train_o, x_test_o, y_train, y_test = train_test_split(features, target, test_size=0.20,random_state=1)

x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_train_mean)/x_train_std

# Reshape image in 3 dimensions (height = 100px, width = 100px , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))

In [14]:
len(x_train)
len(x_test)

6422

In [11]:
model_ig = Sequential()

model_ig.add(Conv2D(16, (3, 3), input_shape = (75, 100, 3),padding='same', activation = 'relu'))
model_ig.add(MaxPooling2D(pool_size = (2, 2)))
model_ig.add(Conv2D(32, (3, 3), padding = 'same',activation = 'relu'))
model_ig.add(MaxPooling2D(pool_size = (2, 2)))
model_ig.add(Conv2D(64, (3, 3), padding = 'same',activation = 'relu'))
model_ig.add(MaxPooling2D(pool_size = (2, 2)))
model_ig.add(Flatten())
model_ig.add(Dense(units = 128, activation = 'relu'))
model_ig.add(Dense(units = 1, activation = 'sigmoid'))

model_ig.summary()
model_ig.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

datagen = ImageDataGenerator(validation_split=0.20)

datagen.fit(x_train)

# fits the model on batches with real-time data augmentation:
training_generator = datagen.flow(x_train, y_train, batch_size=128)

result = model_ig.fit_generator(training_generator, epochs=20)   

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 75, 100, 16)       448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 25, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 12, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6912)             

In [12]:
test_loss, test_acc = model_ig.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)


Test accuracy: 0.9975085854530334


In [13]:
model_ig = Sequential()

model_ig.add(Conv2D(16, (3, 3), input_shape = (75, 100, 3),padding='same', activation = 'relu'))
model_ig.add(MaxPooling2D(pool_size = (2, 2)))
model_ig.add(Conv2D(32, (3, 3), padding = 'same',activation = 'relu'))
model_ig.add(MaxPooling2D(pool_size = (2, 2)))
model_ig.add(Conv2D(64, (3, 3), padding = 'same',activation = 'relu'))
model_ig.add(MaxPooling2D(pool_size = (2, 2)))
model_ig.add(Flatten())
model_ig.add(Dense(units = 128, activation = 'relu'))
model_ig.add(Dense(units = 1, activation = 'sigmoid'))

model_ig.summary()
model_ig.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

datagen = ImageDataGenerator(validation_split=0.20)

datagen.fit(x_train)

training_generator = datagen.flow(x_train, y_train, batch_size=256)

result = model_ig.fit(training_generator, epochs=20)   

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 75, 100, 16)       448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 37, 50, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 37, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 25, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 12, 64)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6912)             